In [ ]:
import os
from libv3.utils import *
import pandas as pd

In [ ]:
############ configuration ################
############################################

CODE, BEHAVIOUR, THREAD, VER = get_config()   ### config stored in libv3/exp_config.txt

base_dir = '../trace_data' ### can be replaced with 'csv', 'exe_plot', 'histogram'
log_path = base_dir+f'/{CODE}/{THREAD}_thread/version_{VER}/{BEHAVIOUR}'

print(log_path)

#### file to display
trace_file = 1

print('file number:', trace_file)

In [ ]:
######### get paths #######################
paths_log, paths_traces, varlist_path, paths_label = get_paths(log_path)

paths_log.sort()
paths_traces.sort()
varlist_path.sort()

print(paths_log)
print(paths_traces)
print(varlist_path)
print(paths_label)

In [ ]:
############# check varlist is consistent ############
############# only for version 3 ######################

if VER == 3:
    to_number = is_consistent(varlist_path)

    if to_number != False:
        from_number = mapint2var(to_number)


In [ ]:
############ Get variable list ######################
var_list = [value for key, value in from_number.items()]   ### get the variable list

## Trace Plots

### Process Traces

In [ ]:
########## process the traces ###########
col_data = preprocess_traces(var_list, paths_traces)   ### in the format (trace_name, x_data, y_data, y_labels, trace_path) 

### Generate plot trace data

In [ ]:
############### preprocessing data to plot using plotly ##############
'''
Restructure the data in dictionary with (keys,value) pair :-  (time, timestamps) , (trace_name, trace)
'''
all_df = get_dataframe(col_data)   

In [ ]:

### plot single trace
for i, df in enumerate(all_df):
    if i == trace_file:
        plot_single_trace(df, var_list, with_time=False, is_xticks=True)

In [ ]:
### get timestamp

timestamp = index2timestamp(all_df[0], 227)
print(timestamp)

## Execution Interval Plots

### Preprocess data

In [ ]:
####### preprocessing data ########

#### extract timestamps for each variable and store them in a dictionary along with index values for each variable in event trace

var_timestamps = get_var_timestamps(paths_traces)    #### in format (filename, dict of timestamps and index values)

to_plot = preprocess_variable_plotting(var_timestamps, var_list, from_number, trace_number=trace_file)   ### restructure the data for plotting


### Generate execution interval plots

In [ ]:
'''
TODO:
- Add option to plot anomalies in trace plotting
- Add option to plot variable exe interval for single trace (added)
- Define format of detected anomalies for plotting (done)
'''

In [ ]:
### plot the data
plot_execution_interval_single(to_plot, is_xticks=False)

## Plot Labels

In [ ]:
### read labels
'''
labels are of format [index1, index2, timestamp1, timestamp2, class]
'''
for i, path in enumerate(paths_label):
    if i == trace_file:
        print(path)
        labels = read_json(path)

########## extract indeices of the labels for plotting ##########
labels_content = labels['labels']
label_trace_name = list(labels_content.keys())[0]
label_content = labels_content[label_trace_name]

label_plot_values = [(x[0], x[1]) for x in label_content]
label_plot_x_ticks = [(x[2], x[3]) for x in label_content]
label_plot_class = [x[4] for x in label_content]

label_content = (label_plot_values, label_plot_x_ticks, label_plot_class)   ####### in format (values, x_ticks(timestamp), class)


In [ ]:
### add all the traces to the graph
for i, df in enumerate(all_df):
    if i == trace_file:
        plot_single_trace(df, var_list, with_time=False, is_xticks=True, anomalies=label_content)